# Notebook to generate indicators module

In [33]:
import project
import inspect

from mintalib import core


In [34]:
def core_functions(first_param=('series', 'prices')):
    """ list of core functions """

    result = []
    
    for k, v in vars(core).items():
        if k.startswith("_") or not k.islower():
            continue

        if isinstance(v, type) or not callable(v):
            continue

        params = list(inspect.signature(v).parameters)

        if params[0] not in first_param:
            continue

        result.append(k)

    return result


print(core_functions())

['calc_avgprice', 'calc_typprice', 'calc_wclprice', 'calc_midprice', 'calc_price', 'crossover', 'crossunder', 'flag_above', 'flag_below', 'invert_flag', 'updown_flag', 'calc_log', 'calc_exp', 'calc_roc', 'calc_diff', 'calc_min', 'calc_max', 'calc_sum', 'calc_mad', 'calc_stdev', 'calc_sma', 'calc_ema', 'calc_rma', 'calc_wma', 'calc_dema', 'calc_tema', 'calc_ma', 'calc_rsi', 'calc_plusdi', 'calc_minusdi', 'calc_adx', 'calc_trange', 'calc_atr', 'calc_natr', 'calc_latr', 'calc_psar', 'calc_cci', 'calc_cmf', 'calc_mfi', 'calc_bop', 'calc_bbands', 'calc_keltner', 'efficiency_ratio', 'calc_kama', 'calc_macd', 'calc_ppo', 'calc_slope', 'calc_curve', 'calc_stoch', 'streak_up', 'streak_down', 'calc_eval']


In [35]:
from mintalib.model import Indicator

class EMA(Indicator):

    same_scale = True

    def __init__(self, period: int, *, item: str=None):
        self.period = period
        self.item = item

    def calc(self, data):
        series = self.get_series(data)
        result = core.calc_ema(series, period=self.period, wrap=True)
        return result


In [36]:
def make_indicator(func, name:str = None, verbose=False):
    if name is None:
        name = func.__name__.upper()
        if name.startswith("CALC_"):
            name = name.partition('_')[2]

    fname = func.__qualname__
    module = func.__module__.rpartition('.')[2]
    qname = f"{module}.{fname}"

    indent = " " * 4
    indent2 = " " * 8

    signature = inspect.signature(func)
    parameters = list(signature.parameters.values())
    Parameter = inspect.Parameter

    if verbose:
        print("signature", signature)
        print("parameters", parameters)

    newparams = []
    extras = dict()
    ftype = parameters[0].name

    def annotate(param):
        if param.name in ['period']:
            return param.replace(annotation=int)

        if param.annotation == 'str':
            return param.replace(annotation=str)

        if type(p.default) in (int, float, bool):
            return param.replace(annotation=type(p.default))
        
        return param

    for i, p in enumerate(parameters):
        if i == 0:
            p = Parameter('self', Parameter.POSITIONAL_OR_KEYWORD)

        if p.name == 'wrap':
            # extras.update(wrap=True)
            continue

        p = annotate(p)
        newparams.append(p)

    if ftype == 'series':
        p = Parameter('item', Parameter.KEYWORD_ONLY, default=None, annotation=str)
        newparams.append(p)

    if verbose:
        print("newparams", newparams)
        print("extras", extras)

    newsig = inspect.Signature(newparams)

    def argument(arg):
        return ftype if arg == 'self' else f"{arg}=self.{arg}"

    args = [argument(p.name) for p in newparams if p.name not in ['item']]
    args += [f"{k}={v!r}" for k, v in extras.items()]
    args = ", ".join(args)

    code = ""
    #code = "# noinspection PyPep8Naming\n"

    code += f"@wrap_function({qname})\n"
    code += f"class {name}:\n"
    
    if len(newparams) > 1:
        code += indent + f"def __init__" + str(newsig) + ":\n"
        for p in newparams[1:]:
            name = p.name
            code += indent2 + f"self.{name} = {name}\n"
        code += "\n"

    if ftype == 'prices':
        code += indent + f"def __calc__(self, prices):\n"
    elif ftype == 'series':
        code += indent + f"def __calc__(self, data):\n"
        code += indent2 + "series = self.get_series(data)\n"

    code += indent2 + f"result = {qname}({args})\n"
    code += indent2 + f"return wrap_result(result, {ftype})\n"

    return code


print(make_indicator(core.calc_trange))

@wrap_function(core.calc_trange)
class TRANGE:
    def __init__(self, *, log_prices: bool = False, percent: bool = False):
        self.log_prices = log_prices
        self.percent = percent

    def __calc__(self, prices):
        result = core.calc_trange(prices, log_prices=self.log_prices, percent=self.percent)
        return wrap_result(result, prices)


In [37]:
def make_indicators():
    buffer = "# Do not edit. File was generated by make-indicators!\n\n"

    buffer += "from . import core\n"
    buffer += "from .utils import get_series, wrap_function, wrap_result\n\n"
    
    buffer += "nan: float = float('NAN')\n"
    buffer += "\n\n"

    names = core_functions()

    for name in names:
        func = getattr(core, name)
        text = make_indicator(func)
        buffer += text + "\n\n"

    buffer += "__all__ = [n for n in dir() if n.isupper()]\n"

    return buffer

code = make_indicators()
print(code[:256], "...")



# Do not edit. File was generated by make-indicators!

from . import core
from .utils import get_series, wrap_function, wrap_result

nan: float = float('NAN')


@wrap_function(core.calc_avgprice)
class AVGPRICE:
    def __calc__(self, prices):
        resu ...


In [38]:
import importlib.util

def new_module(name):
    spec = importlib.util.spec_from_loader(name, None)
    return importlib.util.module_from_spec(spec)

code = make_indicators()

indicators = new_module("mintalib.indicators")

exec(code, indicators.__dict__)

dir(indicators)

['ADX',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BOP',
 'CCI',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'CURVE',
 'DEMA',
 'DIFF',
 'EFFICIENCY_RATIO',
 'EMA',
 'EVAL',
 'EXP',
 'FLAG_ABOVE',
 'FLAG_BELOW',
 'INVERT_FLAG',
 'KAMA',
 'KELTNER',
 'LATR',
 'LOG',
 'MA',
 'MACD',
 'MAD',
 'MAX',
 'MFI',
 'MIDPRICE',
 'MIN',
 'MINUSDI',
 'NATR',
 'PLUSDI',
 'PPO',
 'PRICE',
 'PSAR',
 'RMA',
 'ROC',
 'RSI',
 'SLOPE',
 'SMA',
 'STDEV',
 'STOCH',
 'STREAK_DOWN',
 'STREAK_UP',
 'SUM',
 'TEMA',
 'TRANGE',
 'TYPPRICE',
 'UPDOWN_FLAG',
 'WCLPRICE',
 'WMA',
 '__all__',
 '__annotations__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'core',
 'get_series',
 'nan',
 'wrap_function',
 'wrap_result']

In [39]:
outfile = project.pkgdir / "indicators.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(code)


Updating indicators.py ...


16899